In [154]:
import pandas as pd
import boto3
import spacy
import nltk
!python -m spacy download en_core_web_sm
from nltk.tokenize.treebank import TreebankWordDetokenizer

class SentimentFilter(object):
    
    def __init__(self, text):
        self.text = text
        self.sent_df = None

    def getSentiment(self):
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(self.text)
        sentence_tokens = [sents.text for sents in doc.sents]
        sent_list = []
        pos_list = []
        neg_list = []
        neu_list = []
        mix_list = []

        def sentenceSentiment(text):
            comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
            sentiment_json = comprehend.detect_sentiment(Text=text, LanguageCode='en')
            sent = sentiment_json['Sentiment']
            sent_pos = sentiment_json['SentimentScore']['Positive']
            sent_neg = sentiment_json['SentimentScore']['Negative']
            sent_neu = sentiment_json['SentimentScore']['Neutral']
            sent_mix = sentiment_json['SentimentScore']['Mixed']

            return sent, sent_pos, sent_neg, sent_neu, sent_mix

        for s in sentence_tokens:
            a, b, c, d, e = sentenceSentiment(s)
            sent_list.append(a)
            pos_list.append(b)
            neg_list.append(c)
            neu_list.append(d)
            mix_list.append(e)

        df = pd.DataFrame({'sentence': sentence_tokens, 'sentiment': sent_list, 'pos_prob': pos_list, 
                           'neg_prob': neg_list, 'neutral_prob':neu_list, 'mixed_prob': mix_list })
        df = df.round(2)
        self.sent_df = df

    def filterNeutrals(self):
        """
        Filters out all neutral sentences.
        """
        self.sent_df = self.sent_df[self.sent_df['sentiment'] != 'NEUTRAL']
        self.sent_df.reset_index(inplace=True, drop=True)
        
        
    def filterPositives(self, threshold=0):
        """
        Filters out positive sentiment sentences with score below the threshold.
        """
        self.sent_df = self.sent_df[~((self.sent_df['sentiment'] == 'POSITIVE') &
                                      (self.sent_df['pos_prob'] < threshold))]
        self.sent_df.reset_index(inplace=True, drop=True)
    
    def filterNegatives(self, threshold=0):
        self.sent_df = self.sent_df[~((self.sent_df['sentiment'] == 'NEGATIVE') &
                                      (self.sent_df['neg_prob'] < threshold))]
        self.sent_df.reset_index(inplace=True, drop=True)
    
    def filterMixed(self, threshold=0):
        """
        Filters out mixed sentiment sentences with score below the threshold.
        """
        self.sent_df = self.sent_df[~((self.sent_df['sentiment'] == 'MIXED') &
                                      (self.sent_df['mix_prob'] < threshold))]
        self.sent_df.reset_index(inplace=True, drop=True)
    
    def getDataFrame(self):
        """
        Get the sentiment dataframe.
        """
        return self.sent_df
    
    def getFilteredTokens(self):
        """
        Get the sentence tokens from the text.
        """
        self.filtered_tokens = list(self.sent_df['sentence'])
        return self.filtered_tokens
    
    def getFilteredText(self):
        """
        Get the text that has had sentences filtered out.
        """
        self.filtered_text = TreebankWordDetokenizer().detokenize(list(self.sent_df['sentence']))
        return self.filtered_text

In [248]:
text = "(mellow piano music) (upbeat music) - This is the GoPro MAX. And it's GoPro's second attempt at a consumer level, 360 camera. (camera whirring) Last year, Insta360 came out with the One X, and it totally revolutionized the idea of what a 360 camera could be used for. First off, footage shot with the One X was stitched in camera, and then you were able to edit it right on your phone and upload to social platforms almost instantly. Secondly, they market it not as a 360-degree camera but instead a camera that could shoot 360 degrees of possible angles. Now fast forward a year and we have the GoPro Max, which is the successor to the GoPro Fusion that came out in 2017, that was filled with all of the headaches of 360's past. It required two memory cards. You had to stitch the footage on a desktop. But they overhauled all of that with the Max, and they in turn made a really accessible 360 camera, which makes me believe I might continue to actually use this thing. ♪ We wag, we wag, we wag, we wag, we wag ♪ ♪ This is the widest angle GoPro's made ♪ ♪ This is the wide ♪ The GoPro MAX has dual 180-degree lenses that shoot 16.6-megapixel, 360-degree photos, 5.5-megapixel single lens, or GoPro is calling Max SuperView photos, and the super cool 6.2-megapixel panoramic photos called PowerPanos. Video-wise, you have 1440p60 video with a single lens and 5.6K30 spherical video, just like the Insta360. There are mics on all but one side and it shows the same super satisfying rubber buttons and button layout as the Hero8. On the bottom, you have these new stowable mounting prongs, which alas, you no longer need that housing, which I somehow always manage to forget anyways, and it also makes it compatible with all GoPro mounts. The touchscreen on the Max is a bit smaller than the touchscreen on the back of the Hero, but its menu system is identical. The big plus here is if you're vlogging or taking a selfie, you can see yourself much like on the DJI Osmo Action because, well, there's lenses on both sides. The Max is waterproof up to 16 feet. But that proofing is merely for protection at this point. GoPro claims that it's really difficult to stitch underwater footage, but they are promising some sort of waterproof housing in the future. The screen is super responsive, but you can't scroll around your frame in 360 mode since the screen has swiping gestures such as swiping down to reach the main menu. You can however switch between lenses but only while not recording. Once you hit that record button, the screen has no functionality other than just being a viewfinder to the lens that you chose before rolling. The image on the screen will however stay level with the horizon no matter which way you turn the camera. And it does so with almost no noticeable lag. If you lean into that warped, super fisheye look that only a 360 camera could give you, the footage is sick. Skin tones are true to life and it's not afraid to overexpose the highlights of it. I prefer this to the flatter color grade of the One X, especially for users who won't be color correcting. And how even if you are color correcting, I find that the Max and the One X have very little latitude in the coloring process. Now, if you're only gonna be showing these photos on social platforms that you're gonna look at on a phone, who cares, but for a camera that costs $500, I hope it's going a little further than that. (laughs). The Max's 360° video is stabilized in software. It bobs up and down as you'd expect from walking and it can be really jittery, especially at night. The stitching is most noticeable at the top and the bottom of the frame. And since most of GoPro's mounts typically have a wider base than the camera itself, there's almost always a bit of artifact from the mount in the footage. Over-all though, once you export the 360-degree footage, the stitch is pretty clean. The audio, however, is probably the best I've seen on a camera this size. The wind reduction can add a lot of compression, and enough wind can sound like, well this, (wind blowing) Guys, when I say it's windy out, it's like actually very windy out. I am about to blow over. This might make some really good footage as I fly away. But when you're not in 25-mile an hour wind, it actually reduces the sound of wind while prioritizing voices. So while hiking with Alex, I love that the GoPro picked up on the crickets and footsteps, while also keeping our voices at the forefront no matter where the camera was in space. ♪ Yeah, and I think it's gonna be a long, long time ♪ And then there are those PowerPano photos. I gotta be honest guys, I had a tough time not showing you these photos before this video dropped. I mean first and foremost, they make taking a panoramic photo a lot easier. And you don't have to stand there and slowly and robotically move your phone across the landscape. I was really surprised at how crisp these photos turned out to. Even on a desktop, they still look really cool. The time warp feature in 360 mode, which is when the Max just records the time lapse, is really cool, but it's a total battery killer. I saw my battery drop 15% off a fresh charge while shooting this time warp. And I let it record for all of 10 minutes. I think it would help if the screen would go off a bit sooner while shooting these to save some battery. Otherwise, the battery performance is on par with the Hero8. Even though it is a different battery. So yes, you will need to buy extra spares if you plan on heavy use. I was able to get through a full day of heavy use using two of these batteries. 360 cameras are only accessible though if you can do something with the footage. So the software has to be sound, and GoPro's app makes it really easy to edit this footage. Max uses the same GoPro app as the Heroes. Now, I'm no stranger to keyframes, but I think even a casual non-video director from the verge.com could edit the Max's 360 footage. And I actually enjoyed editing it on the app. GoPro also has a 360 desktop app called GoPro Player, which mirrors the mobile app experience with added codec options at export. You have H.264, ProRes, HUVC, and as far as 360 editing goes, I was blown away by its ease. Okay so there's this whole other mode on the Max called Hero mode. And it's basically just when the camera's using one lens as opposed to two. It has more mic controls and it even boasts GoPro's most stable video called Max HyperSmooth. And it also has the widest field of view called Max SuperView. But I gotta be honest, while it does sound good and the footage is stable, the fact that it maxes out at 1440p60 is noticeable. Especially in how much smoothing is being done when the subject is close to the camera. I also noticed the camera struggled to keep the footage level with the horizon, choosing to instead focus on keeping the footage stable. So when climbing down this trail, which to be fair, is a slanted surface, the horizon kept tilting further and further from level to almost a dizzying point. GoPro has become a household name. I mean they're specced out, durable, tiny, portable beasts of cameras. And the Max is no different. And while the Max has great features such as PowerPano, it also has great hardware and a really good touch screen. But for me, it came down to the ease of use of the app that made me wanna keep using this thing. 360 footage used to mean high-end equipment and tons of headaches and post, and that always pushed me away from using it. But when you put all of that into the form factor of a GoPro, a workflow I've been using for years, it's way more approachable. Now, is all of that enough to make this thing more than a novelty camera that will ultimately end up in my forgotten type bed in my basement? Time will truly tell. But it is the most fun and user-friendly 360 camera I have used today. And I'm excited to see this thing as specced out. (coughing) 4K Hero mode? Please? All right, so for 499, you can pre-order a Max today. They start shipping on October 24th and will be on shelves October 27th. The 360 space is getting kinda nuts"


In [230]:
text ="initially make AirDrop a little easier but it doesn't do anything yet. That feature will come with iOS 13.1 which is scheduled to ship atthe end of the month. And that really brings me to iOS 13 in general which is pretty buggy in all of my iPhone review units so far. I've seen some weird glitches and app crashes during testing. And I've had to restart a couple times just to get things back on track. Apple told me some of these things will get cleaned up in iOS 13.1, but if you're currently happy with your phone I would notrush out to upgrade until we know iOS 13 had settled down, whichshould be soon enough. Assuming Apple cleans up the rough edges of iOS 13, it's basically a bunch of small updates to iOS 12 that make things a little cleaner and simpler. There's a Dark Modenow, the Share Sheet's a little nicer. The Photos app is a little more engaging and Reminders has been redesigned. You can swipe to type on the keyboard now. It's a bunch of stuff like that. But if you have any other recent iPhone you'll be getting all of it aswell shortly because you'll be upgraded to iOS 13. We'll have a full iOS 13 review when it arrives so, keep an eye out for that. So, is it worth upgrading to the iPhone 11? If you've got an iPhone older than the 10 R and it's upgrade time for you, I think the answer is yes. The camera is substantially improved and you'll getvastly better battery life than anything aside from the iPhone 10 R.Those are the things most people care about and Apple's really delivered here. I'd only spend the extra money on the iPhone 11 Pro if you really care about the display. I don't think you're missingout on much by not having a telephoto camera lens to be honest. Now I do, really care about displays, so I'm getting a Pro. But I think most people will be very happy with the iPhone 11. That is, after Apple cleans up iOS 13. If you're looking for an even deeper dive in the the iPhone 11 camera, go check out our iPhone 11 ProReview. It has the same basic camera system as the iPhone 11 and we tested the hell out of it. Then, let me know, if you're upgrading which iPhone are you gonna get? Or, are you gonna wait for something else?"

In [249]:
test = SentimentFilter(text)
test.getSentiment()
test.filterPositives(0.75)
test.filterNegatives(0.75)
test.filterNeutrals()

In [301]:
test.getDataFrame().head()

,sentence,sentiment,pos_prob,neg_prob,neutral_prob,mixed_prob
0,"Secondly, they market it not as a 360-degree c...",NEGATIVE,0.01,0.97,0.02,0.0
1,It required two memory cards.,NEGATIVE,0.01,0.82,0.18,0.0
2,"But they overhauled all of that with the Max, ...",POSITIVE,0.99,0.01,0.01,0.0
3,There are mics on all but one side and it show...,POSITIVE,0.94,0.03,0.03,0.0
4,"On the bottom, you have these new stowable mou...",POSITIVE,0.88,0.00,0.12,0.0


In [244]:
test.getFilteredTokens()

["initially make AirDrop a little easier but it doesn't do anything yet.",
 "I've seen some weird glitches and app crashes during testing.",
 "And I've had to restart a couple times just to get things back on track.",
 "There's a Dark Modenow, the Share Sheet's a little nicer.",
 'You can swipe to type on the keyboard now.',
 "The camera is substantially improved and you'll getvastly better battery life than anything aside from the iPhone 10",
 'But I think most people will be very happy with the iPhone 11.']

In [ ]:
!python -m nltk.downloader stopwords
!python -m nltk.downloader universal_tagset
!python -m spacy download en

In [306]:
import pke
#!python -m nltk.downloader stopwords
#!python -m nltk.downloader universal_tagset
#!python -m spacy download en

class KeyWordExtractor(object):
    def __init__(self, text):
        self.text = text
        self.sent_df = None
        self.kw_tfidf = None
        self.kw_kpminer = None
        self.kw_yake = None
        self.kw_textrank = None
        self.kw_singlerank = None
        self.kw_topicrank = None
        self.kw_tprank = None
        self.kw_positionrank = None
        self.kw_mprank = None
        
    def tfidf(self, n=20):
        extractor = pke.unsupervised.TfIdf()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_tfidf = extractor.get_n_best(n=n)
        
    def kpMiner(self, n=20):
        extractor = pke.unsupervised.KPMiner()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_kpminer = extractor.get_n_best(n=n)
        
    def yake(self, n=20):
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_yake = extractor.get_n_best(n=n)
    
    def textRank(self, n=20):
        extractor = pke.unsupervised.TextRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_textrank = extractor.get_n_best(n=n)
    
    def singleRank(self, n=20):
        extractor = pke.unsupervised.SingleRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_singlerank = extractor.get_n_best(n=n)
        
    def textRank(self, n=20):
        extractor = pke.unsupervised.TextRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_textrank = extractor.get_n_best(n=n)
       
    def topicRank(self, n=20):
        extractor = pke.unsupervised.TopicRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_topicrank = extractor.get_n_best(n=n)
    
    def topicalPageRank(self, n=20):
        extractor = pke.unsupervised.TopicalPageRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_tprank = extractor.get_n_best(n=n)
        
    def positionRank(self, n=20):
        extractor = pke.unsupervised.PositionRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_positionrank = extractor.get_n_best(n=n)

    def multiPartiteRank(self, n=20):
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(self.text, language='en')
        extractor.candidate_selection()
        extractor.candidate_weighting()
        self.kw_mprank = extractor.get_n_best(n=n)

    def allExtractors(self, n=20, pos=None, window=10, normalized=False):
        self.tfidf(n=n)
        self.kpMiner(n=n)
        self.yake(n=n)
        self.textRank(n=n)
        self.singleRank(n=n)
        self.topicRank(n=n)
        self.topicalPageRank(n=n)
        self.positionRank(n=n)
        self.multiPartiteRank(n=n)
        
    def getAllKeyWords(self):
        all_keywords = {'tfidf_keywords': self.kw_tfidf, 'kpminer_keywords': self.kw_kpminer, 
                        'yake_keywords': self.kw_yake, 'textrank_keywords': self.kw_textrank,
                        'singlerank_keywords':self.kw_singlerank, 'topicrank_keywords': self.kw_topicrank, 
                        'topicalpagerank_keywords': self.kw_tprank, 'position_keywords': self.kw_positionrank,
                        'multipartiterank_keywords': self.kw_mprank}
        
        return all_keywords

In [307]:
test_kw = KeyWordExtractor(test.getFilteredText())
test_kw.allExtractors()
test_kw.getAllKeyWords()

{'tfidf_keywords': [('360', 64.6191818101344),
  ('you', 59.15097932185532),
  ('footage', 57.439272720119476),
  ('camera', 48.11980906287147),
  ('being', 37.61918181013441),
  ('gopro', 35.89954545007467),
  ('the footage', 35.89954545007467),
  ('360 camera', 28.719636360059738),
  ('really', 22.300733337889394),
  ('this thing', 21.539727270044803),
  ('gon', 21.539727270044803),
  ('gon na', 21.539727270044803),
  ('the screen', 18.539727270044803),
  ('the camera', 18.539727270044803),
  ('app', 18.539727270044803),
  ('if you', 18.379786357175114),
  ('photos', 16.784839767881337),
  ('screen', 16.03993635429049),
  ('super', 15.539727270044803),
  ('360-degree', 14.359818180029869)],
 'kpminer_keywords': [('360', 193.85754543040323),
  ('footage', 172.31781816035843),
  ('camera', 144.3594271886144),
  ('gopro', 107.69863635022402),
  ('360 camera', 86.15890908017921),
  ('really', 66.90220001366818),
  ('gon', 64.6191818101344),
  ('gon na', 64.6191818101344),
  ('photos', 50